In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../src')
from classes.data.satellite_image import SatelliteImage
from utils.utils import *
from utils.plot_utils import *
from utils.image_utils import *
import utils.mappings as mapps

In [ ]:
import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as im
import re
from pyproj import Transformer
from datetime import date
from scipy.ndimage import label
import time
from tqdm import tqdm
import os
from scipy.spatial.distance import cdist
import numpy as np

In [ ]:
def mask_inversion(image):
    img = image.array.copy()
    img = (img * 255).astype(np.uint8)
    img = img.transpose(1,2,0)

    shape = img.shape[0:2]
    grayscale = np.mean(img, axis=2)

    # Threshold : median of the 4th band
    threshold = np.quantile(grayscale, 0.30)  # only on the 4th band
    black = np.ones(shape, dtype=float)
    white = np.zeros(shape, dtype=float)

    # Creation of the mask : all the infrared prixels below the threshold \
    # will be black and all the infrared prixels above the threshold \
    # will be white.

    mask = np.where(grayscale > threshold, white, black)
    image_mask = np.zeros((shape[0],shape[1],3))
    image_mask[:,:,0] = mask
    image_mask[:,:,1] = mask
    image_mask[:,:,2] = mask
    return image_mask

import os
from PIL import Image
import numpy as np
import io

os.getcwd()
dir ="../train_data-PLEIADES-972-2022"
liste = os.listdir(dir + "/images")
liste_filepath = [ dir + "/images/" + name for name in liste]
filename = liste_filepath[56]

si = SatelliteImage.from_raster(
        filename,
        date = None, 
        n_bands = 3,
        dep = None
    )

#data = mask_inversion(si)
#data = (data*255).astype(np.uint8)

data = si.array
data = data.transpose(1,2,0)

# Convertir le tableau en image
image = Image.fromarray(data)

# Sauvegarder l'image en tant que fichier PNG
image.save("image_exemple_SIFT_paper.png")

plt.imshow(image)

In [ ]:
import cv2

# Read the image
image = cv2.imread('image_exemple_SIFT_paper.png')

desired_width = 250 * 6
desired_height = 250 * 6

# Resize the image using bilinear interpolation
image = cv2.resize(image, (desired_width, desired_height), interpolation=cv2.INTER_LINEAR)

# Apply bilateral filtering
#filtered_image = cv2.bilateralFilter(image, 5, 0.1, 3)
filtered_image = image
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (10,5))
ax1.imshow(filtered_image)
ax2.imshow(image)
plt.show()

In [ ]:
#images en nuances de gris
def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_RGBA2BGRA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_RGB2BGR)

    return plt.imshow(a, **kwargs, cmap= "gray")

#images en couleurs
def cv2_imshow2(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_RGBA2BGRA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_RGB2BGR)

    return plt.imshow(a, **kwargs, cmap= "brg")

In [ ]:
# Display the filtered image
cv2_imshow2(filtered_image)

In [ ]:
import cv2

In [ ]:
# Convert the image to grayscale
gray = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2GRAY)

# Create a SIFT object
sift = cv2.xfeatures2d.SIFT_create(nfeatures = 1500)

# Detect keypoints and compute descriptors
keypoints, descriptors = sift.detectAndCompute(gray, None)

# Draw keypoints on the image
image_with_keypoints = cv2.drawKeypoints(filtered_image, keypoints, None)

# Display the image with keypoints
cv2_imshow2(image_with_keypoints)

print(len(keypoints))

In [ ]:
keypoints_coordinates = np.array([ (int(kp.pt[0]),int(kp.pt[1])) for kp in keypoints])
str_keypoint_coord =  [str(kp_coord) for kp_coord in keypoints_coordinates]
unique_vals,indexes = np.unique(str_keypoint_coord, return_index=True)
keypoints_coordinates = keypoints_coordinates[indexes] 
keypoints = [keypoints[idx] for idx in indexes]
descriptors =  [descriptors[idx] for idx in indexes]

In [ ]:
mat_dist = cdist(keypoints_coordinates, keypoints_coordinates)
seuil_distance = 30
mat_dist[ mat_dist > seuil_distance ] = 0
adjacency_matrix = mat_dist
print(len(mat_dist[mat_dist  >0]))

In [ ]:
list_x = []
list_y = []

#ax.imshow(image_with_keypoints)
for i in range(len(keypoints)):
    for j in range(i+1, len(keypoints)):
        # Check if there is a connection between the points
        if adjacency_matrix[i, j] > 0:
            # Plot a line connecting the points
            #print([keypoints_coordinates[i, 0], keypoints_coordinates[j, 0]], [keypoints_coordinates[i, 1], keypoints_coordinates[j, 1]])
            list_x.append([keypoints_coordinates[i, 0], keypoints_coordinates[j, 0]])
            list_y.append([keypoints_coordinates[i, 1], keypoints_coordinates[j, 1]])
print(len(list_x))

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image_with_keypoints)
for x_coord, y_coord in zip(list_x,list_y):
    ax.plot(x_coord,y_coord, color='red', alpha=0.8)
plt.show()


## selection d'un patch référence

In [ ]:
filename = liste_filepath[35]

si = SatelliteImage.from_raster(
        filename,
        date = None, 
        n_bands = 3,
        dep = None
    )


data = si.array
data = data.transpose(1,2,0)


# Convertir le tableau en image
image = Image.fromarray(data)

image.save("image_reference.png")

import cv2

# Read the image
image = cv2.imread('image_reference.png')

# Apply bilateral filtering
image = cv2.resize(image, (desired_width, desired_height), interpolation=cv2.INTER_LINEAR)

filtered_image_ref = cv2.bilateralFilter(image, 5, 0.1, 3)
plt.imshow(filtered_image_ref)

In [ ]:
patch1blanc = filtered_image_ref[1100:1360,150:350,]
patch2 =filtered_image_ref[1210:1370,610:800,]
fig, ax = plt.subplots()
ax.imshow(patch2)

In [ ]:
gray = cv2.cvtColor(patch2, cv2.COLOR_BGR2GRAY)
# Create a SIFT object
sift = cv2.xfeatures2d.SIFT_create(100)
# Detect keypoints and compute descriptors
keypoints_ref, descriptors_ref = sift.detectAndCompute(gray, None)

# Draw keypoints on the image
image_with_keypoints_ref = cv2.drawKeypoints(patch2, keypoints_ref, None)

# Display the image with keypoints
cv2_imshow2(image_with_keypoints_ref)


In [ ]:
keypoints_ref_coordinates = np.array([ (int(kp.pt[0]),int(kp.pt[1])) for kp in keypoints_ref])
str_keypoint_ref_coord =  [str(kp_coord) for kp_coord in keypoints_ref_coordinates]
unique_vals,indexes = np.unique(str_keypoint_ref_coord, return_index=True)
keypoints_ref_coordinates = keypoints_ref_coordinates[indexes] 
keypoints_ref = [keypoints_ref[idx] for idx in indexes]
descriptors_ref =  [descriptors_ref[idx] for idx in indexes]

## distance entre keypoints de références e tkeypoints de l'image test

In [ ]:
mat_dist_btw_descriptors = cdist(descriptors, descriptors_ref)


In [ ]:
min_dist = np.min(mat_dist_btw_descriptors)
indices_match  = np.where(mat_dist_btw_descriptors == min_dist)
indice_0 = indices_match[0][0]
indice_ref_0 = indices_match[1][0]

list_indices = [indice_0]
list_indices_match_in_test =[indice_ref_0]

remaining_mat = np.delete(mat_dist_btw_descriptors, indice_0, axis=0)

compteur = 0
finish  = False
while not finish:
    
    min_dist_new = np.min(remaining_mat)
    
    indice_ref_new = np.where(mat_dist_btw_descriptors == min_dist_new)[1][0]
    indice_new = np.where(remaining_mat == min_dist_new)[0][0]
        
    if min_dist_new > 2*min_dist:
        print(min_dist,min_dist_new)
        finish = True
    else:
        list_indices.append(indice_new)
        list_indices_match_in_test.append(indice_ref_new)
        remaining_mat = np.delete(remaining_mat, indice_new, axis=0)
        #min_dist = min_dist_new
    



## Prise en compte de la structure "graph"

In [ ]:
selected_indices = [] 
for ind_a, a in enumerate(list_indices):
    for ind_b,b in enumerate(list_indices):
        if (a != b):
        
            point_i = keypoints_coordinates[a]
            point_j = keypoints_coordinates[b]

            E_ij = np.sqrt(np.sum(point_i-point_j)**2)

            indice_k = list_indices_match_in_test[ind_a]
            indice_l = list_indices_match_in_test[ind_b]

            point_k = keypoints_ref_coordinates[indice_k]
            point_l = keypoints_ref_coordinates[indice_l]

            E_kl = np.sqrt(np.sum(point_k-point_l)**2)

            gamma = abs(E_ij-E_kl)

            if gamma < 4 :
                selected_indices.append(a)
                selected_indices.append(b)
selected_indices = np.unique(selected_indices)
len(selected_indices)

In [ ]:
selected_keypoints =[ keypoints[i] for i in selected_indices]
# Draw keypoints on the image
image_with_keypoints = cv2.drawKeypoints(filtered_image,
                                         selected_keypoints,
                                         None)

# Display the image with keypoints
cv2_imshow2(image_with_keypoints)